In [42]:
import requests
import fitz
import pdfplumber
import io
import pandas as pd
import re
import itertools as it
import pymupdf

In [26]:
counties = ['Starke']
county_precincts = {'Starke' : ['wayne3', 'wayne2', 'wayne1', 'washington2', 'washington1', 'railroad', 'oregon3', 'oregon2', 'oregon1', 'northbend', 'jackson', 'davis', 'center6', 'center4', 'center5', 'center3', 'center2', 'center1', 'california2', 'wayne4', 'california1']}
url_root = {'Starke' : 'https://starke.in.gov/wp-content/uploads/2024/11/'}
def precinct_url(county, precinct, precinct_num):
    return url_root[county] + precinct + '-1.pdf'

In [27]:
i = 0
county = counties[i]
print(county)
precincts = county_precincts[county]

Starke


In [28]:
output_file = "../data/in_2024_results/" + county + ".csv"
output_2 = "C:/Users/johnsor/Documents/openelections-data-in/2024/counties/20241105__in__general__" + str.lower(county) + "__precinct.csv"
output_2

'C:/Users/johnsor/Documents/openelections-data-in/2024/counties/20241105__in__general__starke__precinct.csv'

In [29]:
#This cell might only work for Lake County
urls = []
for p in precincts:
    urls.append(precinct_url(county, p, i))
urls[:4] + urls[-2:]

['https://starke.in.gov/wp-content/uploads/2024/11/wayne3-1.pdf',
 'https://starke.in.gov/wp-content/uploads/2024/11/wayne2-1.pdf',
 'https://starke.in.gov/wp-content/uploads/2024/11/wayne1-1.pdf',
 'https://starke.in.gov/wp-content/uploads/2024/11/washington2-1.pdf',
 'https://starke.in.gov/wp-content/uploads/2024/11/wayne4-1.pdf',
 'https://starke.in.gov/wp-content/uploads/2024/11/california1-1.pdf']

In [41]:
url = urls[0]
http = urllib3.PoolManager()
temp = io.BytesIO()
temp.write(http.request("GET", url).data)
doc = pymupdf.open(temp)
page = doc[0]
page

TypeError: bad filename: type(filename)=<class '_io.BytesIO'> filename=<_io.BytesIO object at 0x00000242A9685DF0>.

In [32]:
pages = []
for url in urls:
    http = urllib3.PoolManager()
    temp = io.BytesIO()
    temp.write(http.request("GET", url).data)
    with pdfplumber.open(temp) as pdf:
        pages += [[line['text'] for line in page.extract_text_lines()] for page in pdf.pages]

In [33]:
pages

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [21]:
line = lines[125]
print(re.findall(r'\d+-(\w+\s?\w*)\s+- - -', line))

[]


In [22]:
cols = {'county' : [],
        'precinct' : [],
        'office' : [],
        'district' : [],
        'party' : [],
        'candidate' : [],
        'election_day' : [],
        'absentee' : [],
        'early_voting' : [],
        'votes' : []}
party_convert = {'R' : 'REP',
                 'REP' : 'REP',
                 'D' : 'DEM',
                 'DEM' : 'DEM',
                 'L' : 'LIB',
                 'LIB' : 'LIB',
                 'WTP' : 'WTP',
                 'W' : 'WTP',
                 'NP' : '',
                 'I' : 'IND',
                 'IND' : 'IND'
                }

In [23]:
vote = False #Signifies when a "VOTE FOR ..." line has occurred
for line in lines: #for each line of the page
    if re.search(' EMS ', line):
        election_day = re.findall(r'# Of Election Day (\d+)', line)[0]
        absentee = re.findall(r'# Of Paper Absentee (\d+)', line)[0]
        early_voting = re.findall(r'# Of Walk-In Absentee (\d+)', line)[0]
        votes = re.findall(r'PUBLIC COUNT: (\d+)', line)[0]
        registered = re.findall(r'REGISTERED VOTERS: (\d+)', line)[0]
        meta = True
    elif re.search('- - -', line):
        precinct = re.findall(r'\d+-(\w+\s*\w*\s*\w*)\s+- - -', line)[0]
        precinct = re.sub('\xa0', '', precinct) #Looking at you NEW DURHAM 01 in Laporte
        meta = False
        cols['county'].extend([county, county])
        cols['precinct'].extend([precinct, precinct])
        cols['office'].extend(['Ballots Cast', 'Registered Voters'])
        cols['district'].extend(['',''])
        cols['party'].extend(['',''])
        cols['candidate'].extend(['',''])
        cols['election_day'].extend([election_day,''])
        cols['absentee'].extend([absentee,''])
        cols['early_voting'].extend([early_voting,''])
        cols['votes'].extend([votes, registered])
    elif re.search('VOTES=', line):
        office = re.findall(r'VOTES= [0-9.,]+ (.*)', line)[0]
        if re.search(r'[Senator|Representative] District (\d+)', office):
            office, district = office.split(' District ')
        else:
            district = ''
    elif re.search(r'\d%', line) and not meta: #Lines with percentage numbers that are not metadata are the values we want
        E, A, W, V, _, C = line.split(maxsplit=5) #The line is a row of a table of 6 columns
        if re.search(r'\(\w+\)', C):
            P = party_convert[
                re.findall(r'\((\w+)\)', C)[0]
            ]
            C = re.findall(r'\(\w+\) (.*)', C)[0]
        else:
            P = ''
        cols['county'].append(county)
        cols['precinct'].append(precinct)
        cols['office'].append(office)
        cols['district'].append(district)
        cols['party'].append(P)
        cols['candidate'].append(C)
        cols['election_day'].append(E)
        cols['absentee'].append(A)
        cols['early_voting'].append(W)
        cols['votes'].append(V)
    elif re.search(r'^Straight Party Votes Precinct ID: \d+ PartyName Votes \w+', line):
        #Note, the line ablve skips precincts that had no straight party votes for any party
        office = 'Straight Party'
        straights = re.split(r"\s*(\d+)\s*",re.findall(r'PartyName Votes (.*)', line)[0])[:-1]
        for i in range(len(straights)//2):
            party, votes = straights[2*i:2*i+2]
            P = party_convert[party[0]]
            cols['county'].append(county)
            cols['precinct'].append(precinct)
            cols['office'].append(office)
            cols['district'].append('')
            cols['party'].append(P)
            cols['candidate'].append(party)
            cols['election_day'].append('')
            cols['absentee'].append('')
            cols['early_voting'].append('')
            cols['votes'].append(votes)
        meta = True

In [24]:
df = pd.DataFrame(data=cols)
df

,county,precinct,office,district,party,candidate,election_day,absentee,early_voting,votes
0,Lake,CAL 05,Ballots Cast,,,,195,21,153,369
1,Lake,CAL 05,Registered Voters,,,,,,,771
2,Lake,CAL 05,Public Question-Const Amendment,,,Yes,100,4,73,177
3,Lake,CAL 05,Public Question-Const Amendment,,,No,68,6,59,133
4,Lake,CAL 05,President and Vice-President of the U.S.,,DEM,Harris \ Walz,81,12,90,183
...,...,...,...,...,...,...,...,...,...,...
17557,Lake,WT 10,Lake Superior Court Civil Div Room 6,,,Yes,296,29,522,847
17558,Lake,WT 10,Lake Superior Court Civil Div Room 6,,,No,140,8,240,388
17559,Lake,WT 10,Straight Party,,DEM,Democratic Party,,,,302
17560,Lake,WT 10,Straight Party,,REP,Republican Party,,,,601


In [25]:
df['precinct']

0        CAL 05
1        CAL 05
2        CAL 05
3        CAL 05
4        CAL 05
          ...  
17557     WT 10
17558     WT 10
17559     WT 10
17560     WT 10
17561     WT 10
Name: precinct, Length: 17562, dtype: object

In [26]:
df.to_csv(output_file)

In [27]:
df.to_csv(output_2, index=False, encoding='utf-8')